<div dir="rtl">

## تحلیل و نقد کد قدیمی (Seq2Seq)

* **پیچیدگی وحشتناک:** ببین چقدر کد نوشتیم تا *فقط* ۵ جمله را حفظ کنیم! باید داده‌ها را به `One-Hot` تبدیل می‌کردیم (که حافظه زیادی می‌گیرد)، دو مدل مجزا (یکی برای آموزش، یکی برای تست) می‌ساختیم و یک حلقه `decode` دستی می‌نوشتیم.

* **شکنندگی:** این مدل *فقط* ۵ جمله‌ای را که دیده حفظ کرده. اگر بپرسی "how are you" (که در داده‌ها بود) جواب می‌دهد (که البته در تست ما شکست خورد). اگر بپرسی "who are you" (که نبود) احتمالاً یک پاسخ بی‌ربط یا ناقص می‌دهد، چون هیچ درکی از "معنا" ندارد.

* **مدیریت وضعیت:** کل منطق Seq2Seq در انتقال `encoder_states` به `decoder_lstm` و سپس پاس دادن دستی وضعیت (`h` و `c`) در حلقه تست خلاصه می‌شود. این کار طاقت‌فرسا و مستعد خطا بود.

</div>

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# --- ۱. آماده‌سازی داده‌ها (بخش سخت و خسته‌کننده) ---

# داده‌های آموزشی ما (بسیار کوچک و ساده)
# \t = توکن شروع پاسخ (Start Token)
# \n = توکن پایان پاسخ (End Token)
data_pairs = [
    ('hi', '\thello\n'),
    ('hello', '\thi\n'),
    ('how are you', '\ti am fine\n'),
    ('what is your name', '\ti am a bot\n'),
    ('bye', '\tgoodbye\n'),
]

# جدا کردن پرسش‌ها و پاسخ‌ها
input_texts = [pair[0] for pair in data_pairs]
target_texts = [pair[1] for pair in data_pairs]

# ساخت دیکشنری کاراکترها (Vocabulary)
input_characters = sorted(list(set(''.join(input_texts))))
target_characters = sorted(list(set(''.join(target_texts))))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

# پیدا کردن طولانی‌ترین جمله برای Padding
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('تعداد نمونه‌ها:', len(input_texts))
print('تعداد کاراکترهای ورودی یکتا:', num_encoder_tokens)
print('تعداد کاراکترهای خروجی یکتا:', num_decoder_tokens)
print('طولانی‌ترین جمله ورودی:', max_encoder_seq_length)
print('طولانی‌ترین جمله خروجی:', max_decoder_seq_length)


# ساخت دیکشنری برای تبدیل کاراکتر به ایندکس و برعکس
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

# آماده‌سازی داده‌های آموزشی به فرمت One-Hot (بسیار سنگین اما ساده)
# این همان چیزی است که به مدل می‌دهیم
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(data_pairs):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0

    for t, char in enumerate(target_text):
        # decoder_input_data ورودی به دیکودر است (شامل توکن \t)
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data چیزی است که مدل باید پیش‌بینی کند
            # (یک گام زمانی جلوتر از ورودی دیکودر)
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

# --- ۲. ساخت مدل (بخش اصلی Seq2Seq) ---

latent_dim = 64  # ابعاد حالت پنهان LSTM (کوچک برای این مثال)

# --- مدل Encoder ---
# ورودی انکودر
encoder_inputs = Input(shape=(None, num_encoder_tokens))
# لایه LSTM انکودر
# ما به 'state_h' (حالت پنهان) و 'state_c' (حالت سلول) نیاز داریم
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# ما خروجی انکودر را دور می‌ریزیم و فقط "وضعیت" نهایی را نگه می‌داریم
encoder_states = [state_h, state_c]


# --- مدل Decoder ---
# ورودی دیکودر
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# لایه LSTM دیکودر
# ما این لایه را طوری تنظیم می‌کنیم که در هر گام، خروجی بدهد
# و همچنین وضعیت‌های خودش را برگرداند (برای استفاده در زمان تست)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
# نکته کلیدی: ما 'encoder_states' را به عنوان "وضعیت اولیه" به دیکودر می‌دهیم
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# یک لایه Dense برای تبدیل خروجی LSTM به توزیع احتمال روی کاراکترهای خروجی
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


# --- مدل کامل (برای آموزش) ---
# این مدل، انکودر و دیکودر را به هم وصل می‌کند
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# کامپایل مدل
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


# --- ۳. آموزش مدل ---
# از آنجایی که داده‌ها بسیار کم هستند، سریع آموزش می‌بیند
print("شروع آموزش مدل...")
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=1,
          epochs=150,
          validation_split=0.0) # ما داده ولیدیشن نداریم

print("آموزش تمام شد.")


# --- ۴. ساخت مدل‌های مجزا برای "تست" (Inference) ---
# این بخش گیج‌کننده Seq2Seq است. مدل آموزش با مدل تست فرق دارد.
# ما باید مدل‌ها را جدا کنیم تا بتوانیم پاسخ را "کاراکتر به کاراکتر" تولید کنیم.

# مدل انکودر (برای گرفتن جمله و دادن وضعیت)
encoder_model = Model(encoder_inputs, encoder_states)

# مدل دیکودر (برای گرفتن وضعیت و یک کاراکتر و دادن کاراکتر بعدی)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# ورودی دیکودر در زمان تست (فقط یک کاراکتر در هر لحظه)
decoder_inputs_single = Input(shape=(1, num_decoder_tokens))

# خروجی‌ها و وضعیت‌های جدید دیکودر
decoder_outputs_single, state_h_single, state_c_single = decoder_lstm(
    decoder_inputs_single, initial_state=decoder_states_inputs)
decoder_states_single = [state_h_single, state_c_single]

# خروجی نهایی (پیش‌بینی کاراکتر)
decoder_outputs_single = decoder_dense(decoder_outputs_single)

# مدل نهایی دیکودر برای تست
decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs_single] + decoder_states_single
)


# --- ۵. تابع تست (حلقه گفتگو) ---

def decode_sequence(input_seq):
    # ۱. جمله ورودی را انکود کن تا "وضعیت" اولیه دیکودر بدست آید
    states_value = encoder_model.predict(input_seq)

    # ۲. یک توکن "شروع" (\t) بساز
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.0

    stop_condition = False
    decoded_sentence = ''

    # ۳. حلقه تولید کاراکتر به کاراکتر
    while not stop_condition:
        # ۴. مدل دیکودر را اجرا کن
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # ۵. کاراکتر پیش‌بینی شده را پیدا کن
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]

        # ۶. کاراکتر را به جمله خروجی اضافه کن
        decoded_sentence += sampled_char

        # ۷. شرط پایان: یا به '\n' رسیدی یا طولانی شد
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # ۸. آماده‌سازی ورودی بعدی برای دیکودر
        # کاراکتر فعلی به عنوان ورودی بعدی + وضعیت‌های جدید
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]

    return decoded_sentence.strip() # حذف \n

# --- شروع چت ---
print("\n--- تست مدل (فقط جملات آموزشی کار می‌کنند) ---")
for i in range(len(input_texts)):
    input_seq = encoder_input_data[i: i + 1] # یک جمله را بردار
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('ورودی:', input_texts[i])
    print('پاسخ:', decoded_sentence)

print("\n--- تست ورودی دلخواه ---")
# تست یک ورودی که دقیقاً در داده‌ها بوده
custom_input = "how are you"
# آماده‌سازی ورودی دلخواه
custom_seq = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
for t, char in enumerate(custom_input):
    if char in input_token_index:
        custom_seq[0, t, input_token_index[char]] = 1.0

print('ورودی:', custom_input)
print('پاسخ:', decode_sequence(custom_seq))

# تست یک ورودی که در داده‌ها نبوده (احتمالاً پاسخ بی‌معنی می‌دهد)
custom_input = "who are you"
custom_seq = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
for t, char in enumerate(custom_input):
    if char in input_token_index:
        custom_seq[0, t, input_token_index[char]] = 1.0

print('ورودی:', custom_input)
print('پاسخ:', decode_sequence(custom_seq))

تعداد نمونه‌ها: 5
تعداد کاراکترهای ورودی یکتا: 16
تعداد کاراکترهای خروجی یکتا: 17
طولانی‌ترین جمله ورودی: 17
طولانی‌ترین جمله خروجی: 12


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None, 16)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None, 17)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 64),      │     20,736 │ input_layer[0][0] │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │     20,992 │ input_layer_1[0]… │
│                     │ 64), (None, 64),  │            │ lstm[0][1],       │
│                     │ (None, 64)]       │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 17)  │      1,105 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 42,833 (167.32 KB)

 Trainable params: 42,833 (167.32 KB)

 Non-trainable params: 0 (0.00 B)

شروع آموزش مدل...
Epoch 1/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.0551 - loss: 1.5105
Epoch 2/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0794 - loss: 1.5860
Epoch 3/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2088 - loss: 2.1149
Epoch 4/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1412 - loss: 1.7654
Epoch 5/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1410 - loss: 1.7017 
Epoch 6/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.1562 - loss: 1.7245
Epoch 7/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.2037 - loss: 1.5717
Epoch 8/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 144ms/step - accuracy: 0.1125 - loss: 1.5442
Epoch 9/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.1079 - loss: 1.3610
Epoch 10/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.1498 - loss: 1.5666
Epoch 11/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.2190 - loss: 1.7832
Epoch 12/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - 

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# نام مدل از‌پیش‌آموزش‌دیده
model_name = "microsoft/DialoGPT-medium"

# بارگذاری توکنایزر (برای تبدیل متن به اعداد)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# بارگذاری مدل (برای تولید پاسخ)
model = AutoModelForCausalLM.from_pretrained(model_name)

# متغیری برای ذخیره تاریخچه گفتگو (بسیار مهم برای حفظ زمینه)
chat_history_ids = None

print("ربات آماده است. برای خروج 'exit' را تایپ کنید.")

while True:
    # ۱. دریافت ورودی کاربر
    user_input = input(">> شما: ")
    if user_input.lower() == "exit":
        break

    # ۲. توکنایز کردن ورودی جدید به همراه افزودن توکن پایان جمله
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # ۳. الحاق ورودی جدید به تاریخچه گفتگو
    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
    else:
        bot_input_ids = new_user_input_ids

    # ۴. تولید پاسخ از مدل
    # .generate از پارامترهای مختلفی برای کنترل خلاقیت استفاده می‌کند
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,       # جلوگیری از تکرار کلمات
        do_sample=True,               # فعال‌سازی نمونه‌برداری برای خلاقیت
        top_k=50,                     # انتخاب از ۵۰ کلمه محتمل
        top_p=0.9,                    # انتخاب از کلماتی که مجموع احتمالشون ۹۰٪ است
        temperature=0.7               # کنترل خلاقیت (کمتر = قطعی‌تر، بیشتر = خلاق‌تر)
    )

    # ۵. دیکود کردن پاسخ و چاپ آن
    # ما فقط بخش جدید تولید شده (پاسخ ربات) را می‌خواهیم، نه کل تاریخچه
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print(f"ربات: {response}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

ربات آماده است. برای خروج 'exit' را تایپ کنید.


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


ربات: I didn't even know there was a word for this, lol. Thanks!
ربات: Hey, I'm Tom!
>> شما: exit


<div dir="rtl">
<h2 style="font-size: 36px; font-weight: bold;">مقایسه دو رویکرد: چت‌بات قدیمی در برابر مدرن</h2>
<table border="1" cellpadding="8" style="width:100%; border-collapse: collapse; font-size: 30px; line-height: 1.6;">
  <thead>
    <tr style="background-color: #f0f0f0;">
      <th style="text-align: right; padding: 14px;">ویژگی</th>
      <th style="text-align: right; padding: 14px;">🔴 رویکرد قدیمی (Seq2Seq/Keras)</th>
      <th style="text-align: right; padding: 14px;">🟢 رویکرد مدرن (Transformers/HF)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="text-align: right; padding: 12px;"><b>معماری اصلی</b></td>
      <td style="text-align: right; padding: 12px;">Seq2Seq (Encoder-Decoder با LSTM/RNN)</td>
      <td style="text-align: right; padding: 12px;">Transformer (مبتنی بر Self-Attention)</td>
    </tr>
    <tr>
      <td style="text-align: right; padding: 12px;"><b>پارادایم</b></td>
      <td style="text-align: right; padding: 12px;">🧱 <b>آموزش از صفر (Training from Scratch)</b> <br> (باید الفبا را از اول به او یاد بدهی)</td>
      <td style="text-align: right; padding: 12px;">🧠 <b>استفاده از مدل از‌پیش‌آموزش‌دیده (Pre-trained)</b> <br> (استخدام یک متخصص که قبلاً دکترا گرفته)</td>
    </tr>
    <tr>
      <td style="text-align: right; padding: 12px;"><b>کیفیت پاسخ</b></td>
      <td style="text-align: right; padding: 12px;"><b>بسیار ضعیف و شکننده.</b> <br> (همانطور که دیدی، حتی ۵ جمله را هم یاد نگرفت)</td>
      <td style="text-align: right; padding: 12px;"><b>هوشمند، منسجم و طبیعی.</b> <br> (درک واقعی از زبان و زمینه)</td>
    </tr>
    <tr>
      <td style="text-align: right; padding: 12px;"><b>نیاز به داده (برای نتیجه قابل قبول)</b></td>
      <td style="text-align: right; padding: 12px;"><b>اَبَر-زیاد</b> (میلیون‌ها یا میلیاردها جفت گفتگو)</td>
      <td style="text-align: right; padding: 12px;"><b>صفر</b> (برای استفاده عمومی) <br> (داده‌های کم، فقط برای Fine-tune)</td>
    </tr>
    <tr>
      <td style="text-align: right; padding: 12px;"><b>پیچیدگی کد</b></td>
      <td style="text-align: right; padding: 12px;"><b>بسیار بالا.</b> <br> (آماده‌سازی داده پیچیده، تعریف انکودر، دیکودر، مدل تست مجزا، حلقه پیش‌بینی دستی)</td>
      <td style="text-align: right; padding: 12px;"><b>بسیار پایین.</b> <br> (بارگذاری مدل و توکنایزر در ۲ خط، استفاده از `generate`)</td>
    </tr>
    <tr>
      <td style="text-align: right; padding: 12px;"><b>درک زمینه (Context)</b></td>
      <td style="text-align: right; padding: 12px;"><b>ضعیف و کوتاه‌مدت.</b> <br> (کل ورودی را در یک "بردار فکر" فشرده می‌کند و سریعاً فراموش می‌کند)</td>
      <td style="text-align: right; padding: 12px;"><b>بسیار قوی.</b> <br> (به کل تاریخچه گفتگو نگاه می‌کند و ارتباطات را درک می‌کند)</td>
    </tr>
    <tr>
      <td style="text-align: right; padding: 12px;"><b>نتیجه تست ما (با ۵ جمله)</b></td>
      <td style="text-align: right; padding: 12px;"><b>شکست مطلق!</b> 🤯 <br> (`how are you` -> `i am n`)</td>
      <td style="text-align: right; padding: 12px;"><b>(اجرا نشده، اما اگر اجرا می‌شد)</b> <br> پاسخ‌های هوشمندانه و صحیح بلافاصله.</td>
    </tr>
    <tr>
      <td style="text-align: right; padding: 12px;"><b>زمان راه‌اندازی</b></td>
      <td style="text-align: right; padding: 12px;"><b>ساعت‌ها/روزها.</b> <br> (نصب وابستگی‌های جهنمی، زمان طولانی آموزش)</td>
      <td style="text-align: right; padding: 12px;"><b>دقایق.</b> <br> (`pip install` و دانلود مدل)</td>
    </tr>
  </tbody>
</table>
</div>